In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_train.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/wmal_train_comfort_128.joblib")

In [2]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/comfort_train_processed.csv")

In [3]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["pos"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[0 1 1 ... 1 0 0]
[0 1 1 ... 1 0 0]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[0]


In [4]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow.keras as keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
#att = SeqWeightedAttention()(bi)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-8),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True,return_attention=False) (bi)

app = Multiply() ([bi, att])                      
flat = Flatten()(app)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.Adam(learning_rate=0.00005)
m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])


In [ ]:
m.summary()

In [7]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=10,validation_split=0.15)
#m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_pos_sentipolc_0.73645.h5")

Epoch 1/10
18/18 [==============================] - 5s 93ms/step - loss: 0.6569 - accuracy: 0.6514 - val_loss: 0.5958 - val_accuracy: 0.7731
Epoch 2/10
18/18 [==============================] - 1s 45ms/step - loss: 0.5821 - accuracy: 0.7770 - val_loss: 0.5067 - val_accuracy: 0.8479
Epoch 3/10
18/18 [==============================] - 1s 44ms/step - loss: 0.4791 - accuracy: 0.8647 - val_loss: 0.3986 - val_accuracy: 0.9002
Epoch 4/10
18/18 [==============================] - 1s 58ms/step - loss: 0.3798 - accuracy: 0.8841 - val_loss: 0.3170 - val_accuracy: 0.9002
Epoch 5/10
18/18 [==============================] - 1s 63ms/step - loss: 0.3169 - accuracy: 0.8995 - val_loss: 0.2761 - val_accuracy: 0.9077
Epoch 6/10
18/18 [==============================] - 1s 72ms/step - loss: 0.2745 - accuracy: 0.9123 - val_loss: 0.2528 - val_accuracy: 0.9177
Epoch 7/10
18/18 [==============================] - 1s 61ms/step - loss: 0.2465 - accuracy: 0.9264 - val_loss: 0.2430 - val_accuracy: 0.9202
Epoch 8/10
18

In [8]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/comfort_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/wmal_test_comfort_128.joblib")

In [9]:
# define example
data = dataset_test["pos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 0 0 ... 1 1 0]
[0 0 0 ... 1 1 0]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [10]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [11]:
print(predictions)

[[0.7571837  0.24281628]
 [0.7571837  0.24281628]
 [0.99683666 0.00316327]
 ...
 [0.05369868 0.9463013 ]
 [0.00298287 0.99701715]
 [0.9427493  0.05725071]]


In [12]:
import numpy as np
predictions_vec = np.zeros((1171,4))

In [13]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [14]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [15]:
confidences

[0.7571837,
 0.7571837,
 0.99683666,
 0.99683666,
 0.7450328,
 0.9859881,
 0.87445587,
 0.85536635,
 0.54273385,
 0.83280396,
 0.9995382,
 0.9995382,
 0.83800995,
 0.831912,
 0.97995687,
 0.98838717,
 0.98838717,
 0.98838717,
 0.87874264,
 0.99830365,
 0.55009407,
 0.9736483,
 0.65434664,
 0.99985754,
 0.5114623,
 0.95815194,
 0.7034773,
 0.86372817,
 0.86372817,
 0.82412875,
 0.7754234,
 0.97692966,
 0.9952905,
 0.9952905,
 0.7851796,
 0.97738487,
 0.9965184,
 0.9771577,
 0.79009485,
 0.9495435,
 0.8851659,
 0.92479414,
 0.9147471,
 0.8859923,
 0.961898,
 0.5937947,
 0.9921989,
 0.8398144,
 0.991783,
 0.73881173,
 0.72593194,
 0.99472225,
 0.99472225,
 0.70022327,
 0.86524194,
 0.89421797,
 0.9553152,
 0.9946498,
 0.8955826,
 0.97116226,
 0.910455,
 0.99949145,
 0.9965199,
 0.9965199,
 0.9424835,
 0.99058163,
 0.8550585,
 0.69613194,
 0.995082,
 0.995082,
 0.995082,
 0.995082,
 0.7483635,
 0.9884545,
 0.9494327,
 0.69409555,
 0.6754982,
 0.7559037,
 0.7246905,
 0.9215431,
 0.9887911,


In [16]:
len(labels_to_eval)

1171

In [17]:
dataset_test["predicted_label_pos"] = labels_to_eval
dataset_test["confidence_label_pos"] = confidences

In [18]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["pos"],labels_to_eval,digits=5))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
              precision    recall  f1-score   support

           0    0.92090   0.93202   0.92643       662
           1    0.91018   0.89587   0.90297       509

    accuracy                        0.91631      1171
   macro avg    0.91554   0.91395   0.91470      1171
weighted avg    0.91624   0.91631   0.91623      1171



In [ ]:
dataset_test

In [19]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_pos_abstita.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_AGRITREND_pos_full.csv");